Now that I can predict what the next notes are, how about I do that indefinitely?

In [3]:
import mido
import matplotlib.pyplot as plt
import numpy as np
import time

import smart_loop_test

%matplotlib inline

In [4]:
mf = mido.MidiFile('cycle-test_2020-04-15.mid')

In [5]:
track = mf.tracks[0]

In [6]:
current_time = 0
note_ons = []
notes = []
# The index of each note_on message in track
note_is = []

for i in range(len(track)):
    current_time = current_time + track[i].time
    #print("{}, time: {}".format(track[i], current_time))
    if(track[i].type == "note_on"):
        note_ons.append(current_time)
        notes.append(track[i].note)
        note_is.append(i)

In [7]:
us_per_beat = 500000.0
seconds_per_tick = 1e-6 * us_per_beat / mf.ticks_per_beat

In [17]:
fake_end_i = 50
shifts, next_is = smart_loop_test.get_sequence_self_overlaps(np.array(note_ons[0:fake_end_i], dtype=float))
p = [smart_loop_test.labelled_seq_product(note_ons[:fake_end_i], notes[:fake_end_i], s, fake_end_i) for s in shifts]   

In [18]:
max_prod = np.argmax(p)
period = shifts[max_prod]

In [19]:
next_i = next_is[max_prod]
# \forall j, notes[next_is[max_prod] + j] == notes[i + j]


In [44]:
# play all notes from the start then repeating [repeat_starti, repeat_endi)
def thisisthesongthatneverends(track, repeat_starti, repeat_endi):
    assert(repeat_starti >= 0 and repeat_starti < len(track))
    assert(repeat_endi >= 0 and repeat_endi < len(track))
    i = 0
    length = (repeat_endi - repeat_starti)
    while True:
        #print(i)
        yield track[i]
        i = i + 1
        if i >= repeat_starti:
            #print("i: {}\nrepeat_starti: {}\nrepeat_endi: {}\nlength: {}".format(i, repeat_starti, repeat_endi, length))
            #print("a: {}".format(repeat_starti + ((i - repeat_starti) % length)))
            i = repeat_starti + ((i - repeat_starti) % length)

## Play it

In [12]:
port = mido.open_output('Microsoft GS Wavetable Synth 0')

In [45]:
for message in thisisthesongthatneverends(track, note_is[next_i], note_is[fake_end_i]):
    if(message.is_meta):
        continue
    time.sleep(message.time * seconds_per_tick)
    port.send(message)
    

KeyboardInterrupt: 

That actually seems to work! yay!

Now what?

- Look at what the pure-data interface looks like so we can figure out how to adapt this to it
- Maybe make incremental versions of these algorithms so we can calculate as we go along instead of having to do bulk calculations.